In [1]:
import os
import json
from multiprocessing import Pool
from datetime import datetime
import pandas as pd

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import itertools

In [2]:
data_folder = "./output"
output_folder = "./frames"
geo_shapes_folder = "./geodata"
os.makedirs(output_folder, exist_ok=True)

In [3]:
#fnames = [os.path.join(root, f) for root, _, files in os.walk(geo_shapes_folder) for f in files if f.endswith(())]
#local_data = [list(shpreader.Reader(shape_file).geometries()) for shape_file in fnames]
country_color = "#dbdbdb"
urban_areas_color = "#c2c2c2"
text_color = "black"
border_color = "black"
czech_rep_country_color = "#cecece"
czech_rep_country_color = "#6baf78"

geo_resources = [
    #(
    #    shpreader.natural_earth(resolution = "10m", category='cultural', name="urban_areas"),
    #    {"edgecolor": 'none', "facecolor": urban_areas_color, 'alpha': 0.5, 'zorder': 2}
    #),
]

countries = (
        [c.geometry for c in shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="admin_1_states_provinces")).records() if c.attributes['admin'] != 'Czech Republic'],
        {"edgecolor": border_color, "facecolor": country_color, 'alpha': 1, 'zorder': 1}
        )

czech_rep = (
        [c.geometry for c in shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="admin_1_states_provinces")).records() if c.attributes['admin'] == 'Czech Republic'],
        {"edgecolor": border_color, "facecolor": czech_rep_country_color, 'alpha': 1, 'zorder': 1}
)

data = [(shpreader.Reader(resource).geometries(), styling) for resource, styling in geo_resources] + [countries] + [czech_rep]
cities = [city for city in shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="populated_places")).records() if city.attributes.get('TIMEZONE') and city.attributes['TIMEZONE'].startswith('Europe')]
roads = list(shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="roads")).geometries())

In [4]:

def render_frame(input_filepath: str, output_filepath: str):
    try:
        with open(input_filepath, "r", encoding="utf-8-sig") as infile:
            vehicles_json = json.load(infile)
    except Exception as e:
        print(f"{e} {input_filepath} -> {output_filepath}")
        return

    vehicles = vehicles_json["Vehicles"]
    vehicles_df = pd.DataFrame(vehicles)
    try:
        active_vehicles_df = vehicles_df[vehicles_df.IsInactive==False]
    except Exception as e:
        print(f"{e} {input_filepath} -> {output_filepath}")
        active_vehicles_df = vehicles_df

    timestamp_string_formatted = vehicles_json["LastUpdate"][:19]
    timestamp = datetime.strptime(timestamp_string_formatted, "%Y-%m-%dT%H:%M:%S")
    
    fig = plt.figure(figsize=(12,9), facecolor='white', tight_layout=True)
    ax = plt.subplot(projection=ccrs.PlateCarree())
    plt.title('Aktivní vozidla MHD IDSJMK', loc='left')
    plt.title(timestamp.strftime("%Y-%m-%d %A %H:%M:%S"), loc='right')
    plt.tight_layout()

    # render czech rep

    # render background
    for geometry, styling in data:
        ax.add_geometries(geometry, ccrs.PlateCarree(), **styling)

    # render cities
    ax.scatter([c.geometry.x for c in cities],[c.geometry.y for c in cities], c="black", s=10, alpha=1, zorder=3, marker='s')
    for city in cities:
        pos = city.geometry
        ax.annotate(city.attributes['NAME'], (pos.x+0.03, pos.y+0.03), zorder=4)


    # render vehicles
    if not active_vehicles_df.empty:
        ax.scatter(active_vehicles_df['Lng'], active_vehicles_df['Lat'], c="blue", s=5, alpha=1, zorder=5)

    lat_min = 47
    lat_max = 52
    lng_min = 12
    lng_max = 19
    ax.set_extent([lng_min, lng_max, lat_min, lat_max], crs=ccrs.PlateCarree())
    plt.savefig(output_filepath, dpi=80)
    plt.close(fig)

In [5]:
input_filepaths = sorted([os.path.join(data_folder,f) for f in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, f))])
params = [(infile, os.path.join(output_folder,f"{i:06d}.png")) for i, infile in enumerate(input_filepaths)]

#render_frame(params[1][0], params[1][1])

with Pool() as p:
    p.starmap(render_frame, params)


'DataFrame' object has no attribute 'IsInactive' ./output/2022.01.21 05:44:23.json -> ./frames/000343.png
'DataFrame' object has no attribute 'IsInactive' ./output/2022.01.21 05:45:23.json -> ./frames/000344.png
